In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('30_features.csv')

In [4]:
data = data.drop(columns = ['Unnamed: 0'])

In [5]:
date = pd.read_csv('mydata_for_model.csv', usecols = ['date'])

In [8]:
i_OOT = date[date['date']>="2016-11-01"].index

In [10]:
i_tt = date[(date['date']<"2016-11-01")&(date['date']>'2016-01-14')].index

In [23]:
print(i_OOT[0], i_tt[0])

833507 38511


In [12]:
cols = data.iloc[:,:-1].columns.values

In [14]:
var_use = pd.DataFrame(columns = ['Score', 'variable'])

In [15]:
var_use['variable'] = data.columns

In [38]:
var_use

,Score,variable
0,NaN,fulladdress_count_30
1,NaN,address_count_14
2,NaN,fulladdress_pct_increase_0_to_30
3,NaN,fulladdress_count_0_by_30
4,NaN,address_count_3
5,NaN,fulladdress_count_0_by_7
6,NaN,fulladdress_pct_increase_0_to_7
7,NaN,address_count_1
8,NaN,fulladdress_count_1
9,NaN,fulladdress_homephone_count_30


In [18]:
#OOT_data = data[data['date']>="2016-11-01"]
OOT_data = data.loc[i_OOT,:]

In [19]:
#ttdata = data[data['date']<"2016-11-01"]
ttdata = data.loc[i_tt,:]

In [48]:
ss = StandardScaler()
X = ttdata.iloc[:,:-1]
y = ttdata['fraud_label']
OOTX = OOT_data.iloc[:,:-1]
OOTy = OOT_data['fraud_label']
ss.fit(X)
X = pd.DataFrame(data = ss.transform(X), columns = cols)
OOTX = pd.DataFrame(data = ss.transform(OOTX), columns = cols)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)

In [50]:
def FDR(y_pred, y_true, p):
    y = pd.DataFrame({"yp":y_pred, "yt":y_true})
    y = y.sort_values(by=['yp'], ascending = False)
    nrows = int(round(len(y)*p))
    tot = y['yt'].sum()
    rt = sum(y.iloc[:nrows,1])/tot
    return rt

In [51]:
numlist = [20, 20, 30, 30]
mflist = [10, 5, 10, 5]
RFtrain = []
RFtest = []
RFOOT = []

In [52]:
for num,mf in zip(numlist, mflist):
    col_to_use = list(var_use.iloc[:num,1].values)
    rf = RandomForestClassifier(n_estimators = 10, min_samples_leaf = 3, max_features = mf)
    rf.fit(X_train[col_to_use], y_train)
    RFtrain.append(FDR(pd.DataFrame(rf.predict_proba(X_train[col_to_use].values))[1].values, y_train.values, 0.03))
    RFtest.append(FDR(pd.DataFrame(rf.predict_proba(X_test[col_to_use].values))[1].values, y_test.values, 0.03))
    RFOOT.append(FDR(pd.DataFrame(rf.predict_proba(OOTX[col_to_use].values))[1].values, OOTy.values, 0.03))
    print(f'With {num} variables, select {mf} features, FDR at 3% in training data is {RFtrain[-1]}')
    print(f'With {num} variables, select {mf} features, FDR at 3% in test data is {RFtest[-1]}')
    print(f'With {num} variables, select {mf} features, FDR at 3% in OOT data is {RFOOT[-1]}')

With 20 variables, select 10 features, FDR at 3% in training data is 0.5528210498756382
With 20 variables, select 10 features, FDR at 3% in test data is 0.5271640239147387
With 20 variables, select 10 features, FDR at 3% in OOT data is 0.5125733445096395
With 20 variables, select 5 features, FDR at 3% in training data is 0.5520356067548109
With 20 variables, select 5 features, FDR at 3% in test data is 0.5269040811021575
With 20 variables, select 5 features, FDR at 3% in OOT data is 0.5129924559932942
With 30 variables, select 10 features, FDR at 3% in training data is 0.5989003796308418
With 30 variables, select 10 features, FDR at 3% in test data is 0.5760332726800104
With 30 variables, select 10 features, FDR at 3% in OOT data is 0.5549036043587594
With 30 variables, select 5 features, FDR at 3% in training data is 0.5975913077627962
With 30 variables, select 5 features, FDR at 3% in test data is 0.5760332726800104
With 30 variables, select 5 features, FDR at 3% in OOT data is 0.557

In [53]:
ntlist = [10, 20, 50, 100, 200, 10, 20, 50, 100, 200]
mflist = [5,5,5,5,5,10,10,10,10,10]
RFtrain2 = []
RFtest2 = []
RFOOT2 = []

In [55]:
for nt,mf in zip(ntlist, mflist):
    col_to_use = list(var_use.iloc[:30,1].values)
    rf = RandomForestClassifier(n_estimators = nt, min_samples_leaf = 3, max_features = mf)
    rf.fit(X_train[col_to_use], y_train)
    RFtrain2.append(FDR(pd.DataFrame(rf.predict_proba(X_train[col_to_use].values))[1].values, y_train.values, 0.03))
    RFtest2.append(FDR(pd.DataFrame(rf.predict_proba(X_test[col_to_use].values))[1].values, y_test.values, 0.03))
    RFOOT2.append(FDR(pd.DataFrame(rf.predict_proba(OOTX[col_to_use].values))[1].values, OOTy.values, 0.03))
    print(f'With {nt} trees, select {mf} features, FDR at 3% in training data is {RFtrain2[-1]}')
    print(f'With {nt} trees, select {mf} features, FDR at 3% in test data is {RFtest2[-1]}')
    print(f'With {nt} trees, select {mf} features, FDR at 3% in OOT data is {RFOOT2[-1]}')

With 10 trees, select 5 features, FDR at 3% in training data is 0.5975913077627962
With 10 trees, select 5 features, FDR at 3% in test data is 0.5770730439303353
With 10 trees, select 5 features, FDR at 3% in OOT data is 0.5565800502933781
With 20 trees, select 5 features, FDR at 3% in training data is 0.5979840293232098
With 20 trees, select 5 features, FDR at 3% in test data is 0.5749935014296854
With 20 trees, select 5 features, FDR at 3% in OOT data is 0.5582564962279967
With 50 trees, select 5 features, FDR at 3% in training data is 0.5986385652572326
With 50 trees, select 5 features, FDR at 3% in test data is 0.5760332726800104
With 50 trees, select 5 features, FDR at 3% in OOT data is 0.5574182732606874
With 100 trees, select 5 features, FDR at 3% in training data is 0.5987694724440372
With 100 trees, select 5 features, FDR at 3% in test data is 0.577852872368079
With 100 trees, select 5 features, FDR at 3% in OOT data is 0.5586756077116513
With 200 trees, select 5 features, FDR